In [1]:
import hydra
from tqdm import tqdm
from omegaconf import DictConfig, OmegaConf
import tensorflow as tf
from dataset import Dataset
from model import UCCModel
from logging import getLogger
import os
from train import grad, eval
import re

c:\Users\progu\Downloads\Masters\UCC-TF\cifar10


In [2]:
output_path = "outputs/2024-04-14/05-41-01/"
model_name = "model_weights.h5"
cfg = OmegaConf.load(os.path.join(output_path, ".hydra\\config.yaml"))
args = cfg.args
dataset = Dataset(
        num_instances=args.num_instances,
        num_samples_per_class=args.num_samples_per_class,
        object_arr= [x for x in range(10)],
        ucc_start=args.ucc_start,
        ucc_end=args.ucc_end
        )
optimizer = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)
model = UCCModel(cfg)
model.build(input_shape=())
model.load_weights(filepath=os.path.join(output_path, model_name))

x_train shape: (40000, 32, 32, 3)
40000 train samples
10000 val samples
object0:0, num_train:4000, num_val:1000
object1:1, num_train:4000, num_val:1000
object2:2, num_train:4000, num_val:1000
object3:3, num_train:4000, num_val:1000
object4:4, num_train:4000, num_val:1000
object5:5, num_train:4000, num_val:1000
object6:6, num_train:4000, num_val:1000
object7:7, num_train:4000, num_val:1000
object8:8, num_train:4000, num_val:1000
object9:9, num_train:4000, num_val:1000


In [ ]:
def train(model, optimizer, dataset, args, save_path):
    train_summary_writer = tf.summary.create_file_writer(os.path.join(save_path, "train_logs"))
    val_summary_writer = tf.summary.create_file_writer(os.path.join(save_path, "val_logs"))
    with open(os.path.join(output_path, "train.log")) as file:
        last_line = file.readlines()[-1]
    a = re.search(r"Step.*,", last_line)
    current_step = int(last_line[a.start()+5: a.end()-1])
    logger = getLogger()
    num_steps = args.train_num_steps
    physical_devices = tf.config.list_physical_devices('GPU')
    best_eval_acc = 0
    if physical_devices:
        with tf.device('/device:GPU:0'):
            for i in tqdm(range(current_step, 180000)):

                _, [labels, inputs] = dataset.next_batch_train()
                loss_dict, grads = grad(model, inputs, labels)
                with train_summary_writer.as_default():
                    for key in loss_dict.keys():      
                        tf.summary.scalar(key, loss_dict[key], step=i)
                    train_summary_writer.flush()
                    
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
                if i%10 == 0:
                    eval_loss_dict = eval(model, dataset)
                    with val_summary_writer.as_default():
                        for key in eval_loss_dict.keys():      
                            tf.summary.scalar(key, eval_loss_dict[key], step=i)
                        val_summary_writer.flush()
                    text = (f"Step {i+1}, training: ucc_acc={loss_dict['ucc_acc']} weighted_loss={loss_dict['weighted_loss']} ucc_loss={loss_dict['ucc_loss']} ae_loss={loss_dict['ae_loss']} eval: ucc_acc={eval_loss_dict['ucc_acc']} weighted_loss={eval_loss_dict['weighted_loss']} ucc_loss={eval_loss_dict['ucc_loss']} ae_loss={eval_loss_dict['ae_loss']}")
                    print(text)
                    with open(os.path.join(output_path, "train.log"), "a") as file:
                        file.write(text+"\n")
                    acc = eval_loss_dict['ucc_acc']
                    # if acc>= best_eval_acc:
                    #     best_eval_acc = acc
                    model.save_weights(os.path.join(save_path,"model_weights.h5"), save_format="h5")

: 

In [ ]:
train(model=model, save_path=output_path, dataset=dataset, args=args, optimizer=optimizer)

: 

: 